In [1]:
%load_ext autoreload
%autoreload 2

In [4]:
# Load modules
import os
import sys
import numpy as np
from collections import OrderedDict
from matplotlib import pyplot as plt
from osgeo import gdal
from os.path import splitext
from sklearn.ensemble import RandomForestClassifier

import geopandas as gpd

# Import DEA Notebooks scripts
sys.path.append(os.path.abspath('/g/data/u46/users/ck9738/dea-notebooks/algorithms/'))
#from SpatialTools.py import layer_extent
#from SpatialTools.py import rasterize_vector
#from SpatialTools.py import array_to_geotiff


In [5]:
# Load modules
import datacube
from datacube import Datacube
import os
import sys
import warnings
import pandas as pd
import xarray as xr
import numpy as np
import matplotlib.image as mpimg
from datacube.utils import geometry
from datacube.utils.geometry import CRS
from matplotlib import pyplot as plt
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import export_graphviz
from datacube_stats.statistics import GeoMedian

# # Import DEA Notebooks scripts
sys.path.append(os.path.abspath('/g/data/u46/users/ck9738/dea-notebooks/Algorithms/'))
from ClassificationTools import randomforest_train
from ClassificationTools import randomforest_classify
from ClassificationTools import randomforest_eval
from DEADataHandling import tasseled_cap
from DEADataHandling import load_nbarx
from DEADataHandling import load_sentinel

# For nicer notebook plotting, hide warnings (comment out for real analysis)
warnings.filterwarnings('ignore') 

# Set up datacube instance
dc = datacube.Datacube()


s2aws = Datacube(config='/home/547/ck9738/datacube-s2.conf')

In [27]:
def layer_extent(layer):
    """
    Computes min and max extents from GDAL layer features. Compared to
    built-in ".GetExtent" that always calculates extents on unfiltered layers,
    this allows you to compute extents of features in layers filtered with
    'SetAttributeFilter'. Works for point, line and polygon features, returning
    the extent of feature centroids for lines and polygon datasets.
    
    Last modified: March 2018
    Author: Robbi Bishop-Taylor
    :param layer: 
        Shapefile imported as GDAL layer; e.g.        
        >>> data_source = gdal.OpenEx(train_shp, gdal.OF_VECTOR)
        >>> layer = data_source.GetLayer(0)
    :return: 
        Min and max extent in x and y direction
    """

    # Extract tuples of x, y, z coordinates for each point feature
    point_coords = [feature.geometry().Centroid().GetPoint() for feature in layer]

    # Compute mins and maxes across points for each tuple element
    max_x, max_y, max_z = map(min, zip(*point_coords))
    min_x, min_y, min_z = map(max, zip(*point_coords))

    return min_x, max_x, min_y, max_y

In [51]:
# Working directory
os.chdir("/g/data/u46/users/ck9738/Random_forest")

# List of training files to import. Shapefiles can be either points, lines or polygons, 
# but must be in the same projection system as the remote sensing dataset being analysed. 
# Each file should cover a small enough spatial area so as to not slow dc.load function 
# excessively (e.g. 100 x 100km max)
shapefile_loc = '/g/data/u46/users/ck9738/Random_forest/raw_data/train_trees/trees_train.shp'
train_shp = 'raw_data/train_water/training_data_waternonwater.shp'

In [38]:
data_source = gdal.OpenEx(train_shp, gdal.OF_VECTOR)



In [39]:
layer = data_source.GetLayer(0)

In [40]:
xmin, xmax, ymin, ymax = layer_extent(layer)

In [41]:
print(xmin, xmax, ymin, ymax)

-106200.12797711749 -130981.89461591879 -1280352.9282547852 -1323088.9074965029


In [52]:
project_area = gpd.read_file(shapefile_loc)


In [53]:

#convert the shapefile to GDA94 lat-long coords so we can query dc_load using lat long
project_area['geometry']# = project_area['geometry'].to_crs(epsg=4283)

I am densified (15 elements)


0     POLYGON ((1552954.703809302 -3956606.209839772...
1     (POLYGON ((1548149.616500128 -3955776.54177422...
2     POLYGON ((1551400.134294696 -3956880.597315673...
3     POLYGON ((1551719.751600597 -3955786.278460369...
4     POLYGON ((1551242.97148037 -3956302.640326424,...
5     (POLYGON ((1558204.382398091 -3962885.48175492...
6     POLYGON ((1561991.566409934 -3962654.289976444...
7     POLYGON ((1561101.188795853 -3964004.514343562...
8     POLYGON ((1553359.666670039 -3963063.506655537...
9     POLYGON ((1551080.176933464 -3960482.334928988...
10    POLYGON ((1554301.750043284 -3960579.701790389...
11    POLYGON ((1548027.937495656 -3959669.533303382...
12    POLYGON ((1554417.990158431 -3958479.88907342,...
13    POLYGON ((1552790.575036936 -3958547.741075793...
14    POLYGON ((1556830.241449598 -3961238.029789709...
Name: geometry, dtype: object

In [44]:
project_area

I am densified (15 elements)
I am densified (15 elements)


,Id,name,geometry
0,1,native trees,POLYGON ((149.1548988661191 -35.27033530194367...
1,1,native trees,(POLYGON ((149.1012321909921 -35.2687168076938...
2,3,Urban,"POLYGON ((149.1383356922945 -35.2746522441709,..."
3,3,urban,POLYGON ((149.1402120992323 -35.26451308797323...
4,2,introduced trees,POLYGON ((149.1357690130039 -35.26968941387806...
5,2,introduced trees,(POLYGON ((149.2214855381995 -35.3199746207129...
6,2,introduced trees,POLYGON ((149.2624918300874 -35.31332979779194...
7,1,native trees,POLYGON ((149.2547741577301 -35.32644431025771...
8,4,Industrial,POLYGON ((149.1688472412968 -35.32740992969901...
9,6,water,POLYGON ((149.1401495833744 -35.30714440906041...


In [54]:
x = project_area.total_bounds

In [55]:
x

array([ 1547896.70389986, -3964734.76580407,  1562851.7814637 ,
       -3955213.50731482])

In [56]:
xmin, ymax, xmax, ymin = x

In [57]:
print(xmin, xmax, ymin, ymax)

1547896.7038998567 1562851.7814637013 -3955213.5073148217 -3964734.7658040654
